## pre-requisites for Blind Delegated quantum computing (BDQC):

In the literature, it is common to describe BDQC protocols in the MBQC paradigm, which is why readers need to check the Qiskit textbook for the MBQC tutorial first before proceeding further.

## Formal defintion of Blind Delegated quantum computing (BDQC):

The keyword here is **Blindness**: At the end of the protocol, the server gains no information at all about Alice’s input nor the computation.

There are various protocols to achieve this, some of them assume Alice can perform quantum measurement at the end but cannot prepare qubits, but we will focus on what is known as prepare-and-send Blind delegated quantum computing.

### Description Of the protocol

#### Phase 1: preparation phase

Instead of preparing the resource in the $|+\rangle^{\otimes n} $ state for all qubits like we do in MBQC, Alice applies on each qubit $i$ a local random phase shift $\theta_i$ choosen from $\{ 0, \frac{\pi}{4},\frac{2\pi}{4},..., \frac{7\pi}{4} \}$.

Therefore the state for each qubit becomes $ |{+_{\theta_i} }\rangle= \frac{ |{0}\rangle+e^{i\theta_i } |{1}\rangle }{\sqrt{2}} $

![Scheme of phase 1 of BDQC](images/phase1.png)

The value of the local phase shift $\theta_{i \in \{0...n\}}$ must remain secret only known to alice. Thanks to this local phase shift, the input state is hidden from the server
#### Phase 2: interaction phase

* For each qubit *i* in the cluster:
	
	1. **A:** Select measurement Basis $B_i$ in $ \{ |+_{\delta_i}  \rangle , |-_{\delta_i}   \rangle \}$
		- $ {\delta_i} = {\theta_i} + \pi r_i + \text{computation } {\phi_i}$
		- With $r_i$ <- random in $\{0,1\}$ and kept secret
	2. **A → B:**  Measure qubit i in basis $B_i$
	3. **B → A:**  Outcome of measurement on qubit i as $m_i$
	4. **A:** De-randomize $m_i$ by flipping it if $r_i$=1
	5. **A:** adapts $B_{i+1}$ based on $m_i$


### Security of the protocol:
**Input Blindness**: by shifting the input states arbitrarily with a local hidden phase and “correcting” it later by adapting the measurement basis.

**Output Blindness**: is achieved by shifting the measurement angle by $r_i \pi$ at random. therefore when the server returns the value, it won't know what it was supposed to be.


## Implementing BDQC in Qiskit

The idea here will be to implement the MBQC as seen in the previous MBQC tutorial, but we will add local phase shifts to hide the states.

So let's take identity cluster from the MBQC tutorial

In [49]:
from qiskit import *
import numpy as np

nb_reg=3
q = QuantumRegister(nb_reg)
c = ClassicalRegister(nb_reg) # Changed from 9 to 1
qc = QuantumCircuit(q, c)


## The following individual variables are kept secret (by Alice) from the server
#8
A_hidden_phase= np.random.randint(8, size=(nb_reg)) #between 0 and 7 we will mulptiply by pi/4 later
computation_angle=[0,0,0] # the computation here is simple teleportation, so angles are 0
# 2
random_r = np.random.randint(1, size=(nb_reg)) # 

#### Phase 1: preparation phase

# Alice prepares |+++> state
for i in range(nb_reg):
    qc.h(i)
    qc.p(A_hidden_phase[i] *np.pi/4 ,i)

# Then send the state to the server + send classical instructions for entanglements order
# The server appliez  CZ(1,2)CZ(2,3 on the state recieved as instructed.
for i in range(nb_reg-1):
    qc.cz(i,i+1)

####END Phase 1: preparation phase


##########Measurement
def x_measurement(qbit:int):
    # the function meausures the MSB as the bit on the left (for consistency with the litterature, as opposed to usual order in qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    delta=  computation_angle[qbit] - (A_hidden_phase[qbit] *np.pi/4)  -  (random_r[qbit]  *np.pi)
    qc.p(delta ,qbit)
    qc.h(qbit)
    qc.measure(q[qbit],c[nb_reg-1-qbit])
    
x_measurement(0)

#qubit 0 will give X correction if =1 to qubit 1
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.x(1)        
x_measurement(1)

#qubit 1 will give X correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(1)], 1)):
    qc.x(2)

#qubit 0 will give Z correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.z(2)        


x_measurement(2)

print(qc.draw())
qc.draw(output="mpl")
### Running the simulator to obtain measurement outcome
simulator = Aer.get_backend('aer_simulator_statevector') #qasm simulator
transpiled_circuit = transpile(qc, simulator)
job = execute(qc,simulator,shots = 10000)
result = job.result()
counts = result.get_counts(qc)
print(counts)


         ┌───┐┌─────────┐   ┌──────────┐┌───┐┌─┐                            »
q1064_0: ┤ H ├┤ P(5π/4) ├─■─┤ P(-5π/4) ├┤ H ├┤M├────────────────────────────»
         ├───┤└─┬──────┬┘ │ └──────────┘└───┘└╥┘┌──────────┐┌──────┐┌───┐┌─┐»
q1064_1: ┤ H ├──┤ P(0) ├──■──────■────────────╫─┤0         ├┤ P(0) ├┤ H ├┤M├»
         ├───┤┌─┴──────┴┐        │            ║ │          │└──────┘└───┘└╥┘»
q1064_2: ┤ H ├┤ P(7π/4) ├────────■────────────╫─┤          ├──────────────╫─»
         └───┘└─────────┘                     ║ │          │              ║ »
  c48_0: ═════════════════════════════════════╬═╡  If_else ╞══════════════╬═»
                                              ║ │          │              ║ »
  c48_1: ═════════════════════════════════════╬═╡          ╞══════════════╩═»
                                              ║ │          │                »
  c48_2: ═════════════════════════════════════╩═╡0         ╞════════════════»
                                                └──────────┘    